In [4]:
!pip install ultralytics

In [ ]:
import ultralytics
ultralytics.__version__

In [1]:
import cv2
import os
import pandas as pd
from ultralytics import YOLO
from tracker import Tracker
import time
import pickle

# Get the video file path from the user
video_path = input("Enter the path to the video file: ")

# Check if the file exists
if not os.path.exists(video_path):
    print("The specified video file does not exist. Please check the path.")
    exit()

# Load YOLO model
model = YOLO('yolov8s.pt')

# Open video file
cap = cv2.VideoCapture(video_path)

# Define class list for YOLO
class_list = ['person', 'bicycle', 'car', 'motorcycle', 'airplane', 'bus', 'train', 'truck', 'boat', 
              'traffic light', 'fire hydrant', 'stop sign', 'parking meter', 'bench', 'bird', 'cat', 'dog', 
              'horse', 'sheep', 'cow', 'elephant', 'bear', 'zebra', 'giraffe', 'backpack', 'umbrella', 'handbag', 
              'tie', 'suitcase', 'frisbee', 'skis', 'snowboard', 'sports ball', 'kite', 'baseball bat', 'baseball glove', 
              'skateboard', 'surfboard', 'tennis racket', 'bottle', 'wine glass', 'cup', 'fork', 'knife', 'spoon', 
              'bowl', 'banana', 'apple', 'sandwich', 'orange', 'broccoli', 'carrot', 'hot dog', 'pizza', 'donut', 
              'cake', 'chair', 'couch', 'potted plant', 'bed', 'dining table', 'toilet', 'tv', 'laptop', 'mouse', 
              'remote', 'keyboard', 'cell phone', 'microwave', 'oven', 'toaster', 'sink', 'refrigerator', 'book', 
              'clock', 'vase', 'scissors', 'teddy bear', 'hair drier', 'toothbrush']

# Initialize counters and trackers (Reset previous data)
count = 0
tracker = Tracker()
down = {}
up = {}
counter_down = []
counter_up = []

# Define line positions
red_line_y = 198
blue_line_y = 268
offset = 6

# Create folder for saving frames
if not os.path.exists('detected_frames'):
    os.makedirs('detected_frames')

# Video output setup
fourcc = cv2.VideoWriter_fourcc(*'XVID')
output_video_path = 'output.avi'
out = cv2.VideoWriter(output_video_path, fourcc, 20.0, (1020, 500))

# Get video frame rate
frame_rate = cap.get(cv2.CAP_PROP_FPS)

# Adjust the distance in meters for the lines
real_distance_meters = 50  # Distance between the two lines in meters (adjust as necessary)

while True:
    ret, frame = cap.read()
    if not ret:
        break
    count += 1
    frame = cv2.resize(frame, (1020, 500))

    results = model.predict(frame)
    a = results[0].boxes.data
    a = a.detach().cpu().numpy()
    px = pd.DataFrame(a).astype("float")
    list = []

    for index, row in px.iterrows():
        x1 = int(row[0])
        y1 = int(row[1])
        x2 = int(row[2])
        y2 = int(row[3])
        d = int(row[5])
        c = class_list[d]
        # Track car, truck, motorcycle, and bus
        if 'car' in c or 'truck' in c or 'motorcycle' in c or 'bus' in c:
            list.append([x1, y1, x2, y2])
    bbox_id = tracker.update(list)

    for bbox in bbox_id:
        x3, y3, x4, y4, id = bbox
        cx = int(x3 + x4) // 2
        cy = int(y3 + y4) // 2

        # Check if the vehicle crosses the red line (going down)
        if red_line_y < (cy + offset) and red_line_y > (cy - offset):
            down[id] = time.time()   # current time when vehicle touches the first line
        
        if id in down:
            if blue_line_y < (cy + offset) and blue_line_y > (cy - offset):
                elapsed_time = time.time() - down[id]  # time from red to blue line
                if counter_down.count(id) == 0:
                    counter_down.append(id)

                    # Convert pixel distance to real-world distance using real_distance_meters
                    pixel_distance = abs(blue_line_y - red_line_y)
                    scale_factor = real_distance_meters / pixel_distance  # meters per pixel

                    # Calculate real-world speed (in m/s) and convert to km/h
                    real_distance_travelled = pixel_distance * scale_factor
                    a_speed_ms = real_distance_travelled / elapsed_time  # meters per second
                    a_speed_kh = a_speed_ms * 3.6  # Convert to km/h

                    cv2.circle(frame, (cx, cy), 4, (0, 0, 255), -1)
                    cv2.rectangle(frame, (x3, y3), (x4, y4), (0, 255, 0), 2)  # Draw bounding box
                    cv2.putText(frame, str(id), (x3, y3), cv2.FONT_HERSHEY_COMPLEX, 0.6, (255, 255, 255), 1)
                    cv2.putText(frame, str(int(a_speed_kh)) + 'Km/h', (x4, y4), cv2.FONT_HERSHEY_COMPLEX, 0.8, (0, 255, 255), 2)

        # Check for vehicle going up
        if blue_line_y < (cy + offset) and blue_line_y > (cy - offset):
            up[id] = time.time()
        
        if id in up:
            if red_line_y < (cy + offset) and red_line_y > (cy - offset):
                elapsed1_time = time.time() - up[id]
                if counter_up.count(id) == 0:
                    counter_up.append(id)

                    # Convert pixel distance to real-world distance using real_distance_meters
                    pixel_distance1 = abs(red_line_y - blue_line_y)
                    scale_factor1 = real_distance_meters / pixel_distance1  # meters per pixel

                    # Calculate real-world speed (in m/s) and convert to km/h
                    real_distance_travelled1 = pixel_distance1 * scale_factor1
                    a_speed_ms1 = real_distance_travelled1 / elapsed1_time  # meters per second
                    a_speed_kh1 = a_speed_ms1 * 3.6  # Convert to km/h

                    cv2.circle(frame, (cx, cy), 4, (0, 0, 255), -1)
                    cv2.rectangle(frame, (x3, y3), (x4, y4), (0, 255, 0), 2)
                    cv2.putText(frame, str(id), (x3, y3), cv2.FONT_HERSHEY_COMPLEX, 0.6, (255, 255, 255), 1)
                    cv2.putText(frame, str(int(a_speed_kh1)) + 'Km/h', (x4, y4), cv2.FONT_HERSHEY_COMPLEX, 0.8, (0, 255, 255), 2)

    # Text and line colors
    text_color = (0, 0, 0)  # Black color for text
    yellow_color = (0, 255, 255)  # Yellow color for background
    red_color = (0, 0, 255)  # Red color for lines
    blue_color = (255, 0, 0)  # Blue color for lines

    # Draw the frame and lines
    cv2.rectangle(frame, (0, 0), (250, 90), yellow_color, -1)
    cv2.line(frame, (172, 198), (774, 198), red_color, 2)
    cv2.putText(frame, ('Red Line'), (172, 198), cv2.FONT_HERSHEY_SIMPLEX, 0.5, text_color, 1, cv2.LINE_AA)

    cv2.line(frame, (8, 268), (927, 268), blue_color, 2)
    cv2.putText(frame, ('Blue Line'), (8, 268), cv2.FONT_HERSHEY_SIMPLEX, 0.5, text_color, 1, cv2.LINE_AA)

    # Display the counts of vehicles going down and up
    cv2.putText(frame, ('Going Down - ' + str(len(counter_down))), (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 0.5, text_color, 1, cv2.LINE_AA)
    cv2.putText(frame, ('Going Up - ' + str(len(counter_up))), (10, 60), cv2.FONT_HERSHEY_SIMPLEX, 0.5, text_color, 1, cv2.LINE_AA)
    
    # Calculate total vehicles (sum of up and down)
    total_vehicles = len(counter_up) + len(counter_down)
    
    # Display the total vehicle count
    cv2.putText(frame, ('Total Vehicles - ' + str(total_vehicles)), (10, 90), cv2.FONT_HERSHEY_SIMPLEX, 0.5, text_color, 1, cv2.LINE_AA)

    # Save frame to disk
    frame_filename = f'detected_frames/frame_{count}.jpg'
    cv2.imwrite(frame_filename, frame)

    # Write frame to output video
    out.write(frame)

    # Show the frame
    cv2.imshow("frames", frame)
    
    if cv2.waitKey(1) & 0xFF == 27:  # Escape key to exit
        break

# Print final stats and output video path
print(f"Vehicles going down: {len(counter_down)}")
print(f"Vehicles going up: {len(counter_up)}")
print(f"Total vehicles: {len(counter_up) + len(counter_down)}")
print(f"Output video saved to: {output_video_path}")

# Save tracking data to pickle file
tracking_data = {
    'down': down,
    'up': up,
    'counter_down': counter_down,
    'counter_up': counter_up
}

with open('tracking_data.pkl', 'wb') as f:
    pickle.dump(tracking_data, f)

print("Tracking data has been saved to 'tracking_data.pkl'")

# Release resources
cap.release()
out.release()
cv2.destroyAllWindows()


Enter the path to the video file:  highway_mini.mp4



0: 320x640 9 cars, 3 trucks, 483.4ms
Speed: 4.0ms preprocess, 483.4ms inference, 1.8ms postprocess per image at shape (1, 3, 320, 640)

0: 320x640 9 cars, 2 trucks, 206.5ms
Speed: 4.7ms preprocess, 206.5ms inference, 1.1ms postprocess per image at shape (1, 3, 320, 640)

0: 320x640 9 cars, 2 trucks, 309.3ms
Speed: 3.3ms preprocess, 309.3ms inference, 2.5ms postprocess per image at shape (1, 3, 320, 640)

0: 320x640 9 cars, 2 trucks, 403.3ms
Speed: 1.8ms preprocess, 403.3ms inference, 1.2ms postprocess per image at shape (1, 3, 320, 640)

0: 320x640 9 cars, 2 trucks, 204.4ms
Speed: 3.3ms preprocess, 204.4ms inference, 1.1ms postprocess per image at shape (1, 3, 320, 640)

0: 320x640 9 cars, 2 trucks, 91.8ms
Speed: 1.1ms preprocess, 91.8ms inference, 0.7ms postprocess per image at shape (1, 3, 320, 640)

0: 320x640 9 cars, 2 trucks, 85.2ms
Speed: 1.5ms preprocess, 85.2ms inference, 0.6ms postprocess per image at shape (1, 3, 320, 640)

0: 320x640 9 cars, 2 trucks, 79.0ms
Speed: 1.1ms pr